In [1]:
import torch                     
import torch.nn as nn            
import torch.nn.functional as F
torch.set_float32_matmul_precision('high')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [12]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 6, 5)
        self.fc = nn.LazyLinear(10)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = torch.flatten(x,1)
        x = F.relu(self.fc(x))
        return x

In [13]:
!rm -rf torch_compile_debug
model = SimpleModel().to(device)
input_tensor = torch.randn(1,1,32,32).to(device)

model(input_tensor)
compiled_model = torch.compile(model, backend="inductor", 
                       options={'trace.graph_diagram':True,
                                'trace.enabled':True})


out = compiled_model(input_tensor).sum().backward()

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
[2023-03-18 08:17:43,035] torch._inductor.debug: [WARNING] model__2_forward_7 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_08_07_50_087949-pid_825/aot_torchinductor/model__2_forward_7.4
[2023-03-18 08:17:43,095] torch._inductor.debug: [WARNING] model__2_backward_8 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_08_07_50_087949-pid_825/aot_torchinductor/model__2_backward_8.5


Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_08_07_50_087949-pid_825/aot_torchinductor/model__2_forward_7.4/graph_diagram.svg
Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_08_07_50_087949-pid_825/aot_torchinductor/model__2_backward_8.5/graph_diagram.svg


In [15]:
aten = torch.ops.aten

In [17]:
aten.convolution

TypeError: '_OpNamespace' object is not callable

In [19]:
import triton
import triton.language as tl
from torch._inductor.ir import ReductionHint
from torch._inductor.ir import TileHint
from torch._inductor.triton_ops.autotune import pointwise
from torch._inductor.utils import instance_descriptor

@pointwise(size_hints=[8192], filename='test', meta={'signature': {0: '*fp32', 1: '*fp32', 2: '*fp32', 3: '*i1', 4: 'i32'}, 'device': 0, 'constants': {}, 'mutated_arg_names': [], 'configs': [instance_descriptor(divisible_by_16=(0, 1, 2, 3, 4), equal_to_1=())]})
@triton.jit
def triton_(in_ptr0, in_ptr1, out_ptr0, out_ptr1, xnumel, XBLOCK : tl.constexpr):
    xnumel = 4704
    xoffset = tl.program_id(0) * XBLOCK
    xindex = xoffset + tl.arange(0, XBLOCK)[:]
    xmask = xindex < xnumel
    x0 = xindex
    x2 = (xindex // 784)
    tmp0 = tl.load(in_ptr0 + (x0), xmask)
    tmp1 = tl.load(in_ptr1 + ((x0 // 784)), xmask)
    tmp4 = tl.load(in_ptr1 + (x2), xmask)
    tmp2 = tmp0 + tmp1
    tmp3 = tl.where(0 != 0, 0, tl.where(0 > tmp2, 0, tmp2))
    tmp5 = tmp0 + tmp4
    tmp6 = tl.where(0 != 0, 0, tl.where(0 > tmp5, 0, tmp5))
    tmp7 = 0.0
    tmp8 = tmp6 <= tmp7
    tl.store(out_ptr0 + (x0 + tl.zeros([XBLOCK], tl.int32)), tmp3, xmask)
    tl.store(out_ptr1 + (x0 + tl.zeros([XBLOCK], tl.int32)), tmp8, xmask)

In [21]:
from torch._inductor.codecache import AsyncCompile
async_compile = AsyncCompile()

triton__0 = async_compile.triton('''
import triton
import triton.language as tl
from torch._inductor.ir import ReductionHint
from torch._inductor.ir import TileHint
from torch._inductor.triton_ops.autotune import pointwise
from torch._inductor.utils import instance_descriptor

@pointwise(size_hints=[8192], filename=__file__, meta={'signature': {0: '*fp32', 1: '*fp32', 2: '*fp32', 3: '*i1', 4: 'i32'}, 'device': 0, 'constants': {}, 'mutated_arg_names': [], 'configs': [instance_descriptor(divisible_by_16=(0, 1, 2, 3, 4), equal_to_1=())]})
@triton.jit
def triton_(in_ptr0, in_ptr1, out_ptr0, out_ptr1, xnumel, XBLOCK : tl.constexpr):
    xnumel = 4704
    xoffset = tl.program_id(0) * XBLOCK
    xindex = xoffset + tl.arange(0, XBLOCK)[:]
    xmask = xindex < xnumel
    x0 = xindex
    x2 = (xindex // 784)
    tmp0 = tl.load(in_ptr0 + (x0), xmask)
    tmp1 = tl.load(in_ptr1 + ((x0 // 784)), xmask)
    tmp4 = tl.load(in_ptr1 + (x2), xmask)
    
    
    tmp2 = tmp0 + tmp1
    tmp3 = tl.where(0 != 0, 0, tl.where(0 > tmp2, 0, tmp2))
    tmp5 = tmp0 + tmp4
    tmp6 = tl.where(0 != 0, 0, tl.where(0 > tmp5, 0, tmp5))
    tmp7 = 0.0
    tmp8 = tmp6 <= tmp7
    tl.store(out_ptr0 + (x0 + tl.zeros([XBLOCK], tl.int32)), tmp3, xmask)
    tl.store(out_ptr1 + (x0 + tl.zeros([XBLOCK], tl.int32)), tmp8, xmask)
''')

In [27]:
triton__0.run()

AttributeError: 'TritonFuture' object has no attribute 'run'